# Домашнее задание 2. NER
## Таня Казакова

Смотрим укулеле
3+ способа найти упоминания товаров в отзывах:
1. Построить word2vec, посмотреть самые близкие слова к слову укулеле, посмотреть в словарях синонимов синонимы к слову укулеле, добавить все названия марок, типов укулеле из каталога, найти эти слова в тексте в разных падежах.

2. (хорошо идёт вместе с 1) Написать правила с помощью yargy или чего-то подобного.

а) Описать несколько способов упомянуть товар: ADJ ukulele NUM и некоторые другие, где внутри нужное нам слово, а окружение подчиняется правилам.

б) Взять конструкции play the X и некоторые другие, искать X.

3. Обучить нейросети классифицировать каждое слово упоминание/неупоминание, для обучения использовать размеченный корпус.

In [1]:
pip install spacy==3.2

     |████████████████████████████████| 6.0 MB 4.8 MB/s 
     |████████████████████████████████| 451 kB 46.0 MB/s 
     |████████████████████████████████| 181 kB 55.1 MB/s 
     |████████████████████████████████| 10.1 MB 59.3 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 628 kB 54.6 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [52]:
pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 10.9 MB/s 


In [53]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from tqdm import tqdm
tqdm.pandas()
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import spacy

In [3]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 399 kB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import en_core_web_sm

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
### load the meta data

meta_data = []
with gzip.open('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/HW2_meta_Musical_Instruments.json.gz') as f:
    for l in f:
        prod_inf = json.loads(l.strip())
        if 'Ukuleles' in prod_inf['category']:
            meta_data.append(prod_inf)
    
# total length of list, this number equals total number of products
print(len(meta_data))

# first row of the list
# print(meta_data[0])

1667


In [9]:
meta_df = pd.DataFrame.from_dict(meta_data)
meta_df

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Ukuleles, Mandolins &amp...",,"[From Alfred Music, the worldwide leader in mu...",,"Alfreds Teach Yourself to Play Ukulele, Comple...",[],,Alfred Music Publishing,[High-quality Firebrand wood soprano ukulele w...,"[>#24,974 in Musical Instruments (See Top 100 ...","[B01F543PAW, B076KFB49J, B015XD4YLY, B01LYBZ4M...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","July 10, 2011",,0739079891,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Ukuleles, Mandolins &amp...",,[The ukelele has been responsible for launchin...,,Hohner HU-212 Standard Ukulele Nato Fretboard,[],,Lanikai,"[Nato top, back, sides, and neck, 12 frets and...","[>#644,571 in Toys &amp; Games (See Top 100 in...",[],Toys &amp; Games,"class=""a-bordered a-horizontal-stripes a-spa...",,$5.24,B00004WGMB,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
2,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,"[The Uke has a nato top, back and sides... wit...",,Samick UK-50 Tenor Concert-size Ukulele,[],,Samick,[Samick UK-50 Tenor Concert-size Ukulele],"[>#494,306 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 13, 2006",,B0002D02IA,[],[],{}
3,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,[Rogue Baritone Ukulele: The Rogue Baritone Uk...,,Rogue Baritone Ukulele,[],,,"[White wood body, Geared tuners, It has 18 fre...","[>#372,072 in Musical Instruments (See Top 100...",[],Musical Instruments,,"November 6, 2008",,B0002F6NYK,[],[],{}
4,"[Musical Instruments, Ukuleles, Mandolins &amp...",,[Johnson ukuleles are superbly built from fine...,,Johnson Hawaiian Soprano Ukulele,[],,Johnson,"[Mahogany Body, Champion-Style Friction Pegs, ...","[>#238,052 in Musical Instruments (See Top 100...","[B015XD4YLY, B075CRWQ4T, B001LU1SFO, B00172UVG...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","August 7, 2005",,B0002F7H0E,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,[This 24&quot; ukulele of deadwoodrare materia...,,ammoon 24&quot; DeadwoodRare MaterialUkulele w...,[],,ammoon,"[24"" size deadwoodrare material) ukulele., Wit...","[>#116,923 in Musical Instruments (See Top 100...","[B01LYCR3CI, B077ZMJ1SW, B010DEZ2H8, B01HEQ50A...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","September 28, 2016",$14.99,B01HEQ51ZG,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
1663,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,[<b>Specifications:</b><br /> Size: 21&quot;<b...,,ammoon 21&quot; Acoustic Ukulele 15 Fret 4 Str...,[],,ammoon,"[21"" size, suitable for both kids and adults.,...","[>#166,841 in Musical Instruments (See Top 100...","[B010DEZ2H8, B010DEU45I, B06XF4QRP1, B0716PFB1...",Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","September 28, 2016",$5.66,B01HEQ53OA,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
1664,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,[Save With This Bundle : - Kala KA-ZCT-T Ziric...,,Kala KA-ZCT-T Tenor Gloss Ziricote Ukulele w/ ...,[B01H74YXNQ],,Kala,"[Includes a Bag and Tuner!, Binding: Black, Ne...","[>#153,563 in Musical Instruments (See Top 100...",[],Musical Instruments,"class=""a-bordered a-horizontal-stripes a-spa...","June 22, 2016",$55.09,B01HFKFWDW,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,{}
1665,"[Musical Instruments, Ukuleles, Mandolins & Ba...",,[K

In [10]:
len(set(meta_df['asin'].to_list())) # - такие id product

1650

In [11]:
### load the data
rew_data = []
with gzip.open('/content/drive/MyDrive/ВШЭ/Прог-е/python-personal/4 курс/HW2_Musical_Instruments.json.gz') as f:
    for l in f:
        prod_rew = json.loads(l.strip())
        if prod_rew['asin'] in set(meta_df['asin'].to_list()):
            rew_data.append(prod_rew)
    
# total length of list, this number equals total number of products
print(len(rew_data))

# first row of the list
print(rew_data[0])

32337
{'overall': 4.0, 'vote': '25', 'verified': False, 'reviewTime': '12 26, 2011', 'reviewerID': 'A14X6A879W6GEA', 'asin': '0739079891', 'reviewerName': 'Shop Around', 'reviewText': "I hesitated at first, but nabbed the kit on a lightning deal. The uke is pretty easy to learn, even at 30 years old with little music education. The cd and dvd are easy to follow along with. I give four stars because after looking online I saw the kit wasn't that good a discounted lightning deal. Alot of cool ukes out there for the same price that I would have bought instead, but i'm still glad I bought this one with the kit for a beginner.", 'summary': 'Nice Beginner Kit', 'unixReviewTime': 1324857600}


In [12]:
rew_data = pd.DataFrame.from_dict(rew_data)
rew_data

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,image,style
0,4.0,25,False,"12 26, 2011",A14X6A879W6GEA,0739079891,Shop Around,"I hesitated at first, but nabbed the kit on a ...",Nice Beginner Kit,1324857600,NaN,NaN
1,1.0,NaN,False,"01 24, 2017",A3PLX7GLR683LV,0739079891,Brian Michaud,Plain and simple...some of the frets are not p...,Doesn't play in tune - cheaply made,1485216000,NaN,NaN
2,5.0,NaN,True,"01 10, 2017",A1PSHXL9ZSFD4X,0739079891,thomas e keller,"she loved it, won't stop playing",Five Stars,1484006400,NaN,NaN
3,5.0,NaN,True,"12 17, 2016",A253W2LUG7PJWT,0739079891,Jameson Evans,Thank you,Five Stars,1481932800,NaN,NaN
4,4.0,NaN,True,"12 17, 2016",A23QUBH9Q4ZY84,0739079891,Trevor MacDonald,I decided that I wanted to learn to play the u...,Good and cheap for beginners,1481932800,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
32332,5.0,NaN,False,"03 7, 2018",A1EPAJAGKLEARI,B01HFKFWDW,ikbennate,I got this uke as a stand alone; it sounds ama...,Beautiful sound and look.,1520380800,NaN,NaN
32333,3.0,NaN,True,"09 13, 2017",A16CD5M7QS7IZT,B01HFKFWDW,Michael ngai,This is what I want,It's a beautiful eku,1505260800,NaN,NaN
32334,5.0,NaN,True,"02 12, 2017",A33K2TFQLM7VVJ,B01HFKFWDW,Linda,The Kala KA-ZCT Tenor Gloss Ziricote uke sound...,Great tenor ukulele,1486857600,NaN,NaN
32335,4.0,NaN,True,"03 17, 2017",AGGNK5AMUQ0EZ,B01HFKE7RO,kyoung,This little ukulele has a booming voice! It t...,Looks beautiful,1489708800,NaN,NaN


# Составляем словарь сущностей и характеристик
У слова укулеле не так много синонимов. Посмотрим глазами, как их обычно называют.

In [9]:
#rew_data[['reviewText',	'summary']][:100]

In [13]:
meta_df[['brand', 'title', 'feature', 'description', 'category']][:100]

,brand,title,feature,description,category
0,Alfred Music Publishing,"Alfreds Teach Yourself to Play Ukulele, Comple...",[High-quality Firebrand wood soprano ukulele w...,"[From Alfred Music, the worldwide leader in mu...","[Musical Instruments, Ukuleles, Mandolins &amp..."
1,Lanikai,Hohner HU-212 Standard Ukulele Nato Fretboard,"[Nato top, back, sides, and neck, 12 frets and...",[The ukelele has been responsible for launchin...,"[Musical Instruments, Ukuleles, Mandolins &amp..."
2,Samick,Samick UK-50 Tenor Concert-size Ukulele,[Samick UK-50 Tenor Concert-size Ukulele],"[The Uke has a nato top, back and sides... wit...","[Musical Instruments, Ukuleles, Mandolins & Ba..."
3,,Rogue Baritone Ukulele,"[White wood body, Geared tuners, It has 18 fre...",[Rogue Baritone Ukulele: The Rogue Baritone Uk...,"[Musical Instruments, Ukuleles, Mandolins & Ba..."
4,Johnson,Johnson Hawaiian Soprano Ukulele,"[Mahogany Body, Champion-Style Friction Pegs, ...",[Johnson ukuleles are superbly built from fine...,"[Musical Instruments, Ukuleles, Mandolins &amp..."
...,...,...,...,...,...
95,Kohala,Kohala Soprano Ukulele,"[Soprano Size, 12 Frets, One Year Warranty, Ea...",[This is da KineO uke you take if youre going ...,"[Musical Instruments, Ukuleles, Mandolins & Ba..."
96,Oscar Schmidt,"Oscar Schmidt UB1 Soprano Ukulele Gig Bag, Black",[The durable Oscar Schmidt UB1 Soprano Ukulele...,[Uke Soprano Bag],"[Musical Instruments, Ukuleles, Mandolins & Ba..."
97,Lanikai,Lanikai LU-6 6 String Tenor Ukulele,"[Tenor Sized 6 String Ukulele, 18 Frets, Easy ...",[The LU-6 6 String Tenor Ukulele is a favorite...,"[Musical Instruments, Ukuleles, Mandolins & Ba..."
98,Lanikai,Lanikai LU-21B Baritone Ukulele,"[Baritone Sized, 18 Frets, Easy Playablity, Di...",[The Lu-21B is the best selling Baritone Ukule...,"[Musical Instruments, Ukuleles, Mandolins & Ba..."


In [14]:
brands = list(set(meta_df['brand'].to_list()))
# Добавим части брендовых названиий без "укулеле", "гитары". Вручную быстрее.
brands.extend(['Bill Tapia', 'BugsGear', 'Dean', 'Godin', 'Hawaiian Resonator',
              'Islander', 'Kona', 'LANIKAI PRO CUSTOM TENOR', 'Liyra Handcrafted',
              'Liyra', 'Luna', 'Moku', 'Myers', 'Naneki', 'Oriolo', 'Ortega', 'Outdoor',
              'Pono', 'Pukanala', 'Stone Case', 'Toy', 'ChillChill', 'Woods', 'mLaval'])

In [15]:
uknames = ['ukulele', 'ukuleles', 'uke', 'ukes', 'eku', 'ekus' 'instrument', 'it', 'Ukulele', 'Ukuleles', 'Uke', 'Ukes']
nabornames = ['it', 'kit', 'kits']
# Добавим из википедии типов укулеле https://en.wikipedia.org/wiki/Ukulele.
# Почти все (будем думать, что все) могут сочетаться со списком uknames, а могут упоминанться отдельно.
wiki_uktypes = ['pocket', 'piccolo', 'sopranino', 'sopranissimo',
             'soprano', 'standard',
             'concert', 'alto',
             'tenor', 'taro patch', 'liliu',
             'baritone', 'bari', 'bari uke', 'taropatch',
             'bass', 'U-Bass', 'Rumbler',
             'lute', 'string']
# Добавим слова с сайта синонимов
uknames.extend(['cavaquinho', 'machete', 'rajão', 'cavaco', 'cavacolele', 'timple', 'small guitar'])

In [16]:
ukfeatures = []
usual_posfeatures = ['NOUN', 'ADJ', 'PROPN', 'NUM']

for index, row in meta_df.iterrows():
    if 'kulele' in '. '.join(row['feature']):
        doc = nlp('. '.join(row['feature']))
        for tok_id in range(len(doc)):
            if doc[tok_id].lemma_ in uknames:
                try:
                    if doc[tok_id-1].pos_ in usual_posfeatures:
                        if doc[tok_id-2].pos_ in usual_posfeatures or doc[tok_id-2].text == '-':
                            if doc[tok_id-3].pos_ in usual_posfeatures or doc[tok_id-3].text == '-':
                                if doc[tok_id-4].pos_ in usual_posfeatures or doc[tok_id-4].text == '-':
                                    if doc[tok_id-5].pos_ in usual_posfeatures or doc[tok_id-5].text == '-':
                                        if doc[tok_id-6].pos_ in usual_posfeatures:
                                            ukfeatures.append(doc[tok_id-6:tok_id].text)
                                        else:
                                            ukfeatures.append(doc[tok_id-5:tok_id].text)
                                    else:
                                        ukfeatures.append(doc[tok_id-4:tok_id].text)
                                else:
                                    ukfeatures.append(doc[tok_id-3:tok_id].text)
                            else:
                                ukfeatures.append(doc[tok_id-2:tok_id].text)
                        else:
                            ukfeatures.append(doc[tok_id-1].text)
                except Exception:
                    answer = 'Там нет слова'

print(len(ukfeatures))
ukfeatures_set = set(ukfeatures)
print(len(ukfeatures_set))

852
388


In [17]:
ukfeatures_form_titles = []
usual_posfeatures = ['NOUN', 'ADJ', 'PROPN', 'NUM']

for index, row in meta_df.iterrows():
    if 'kulele' in row['title']:
        doc = nlp(row['title'])
        for tok_id in range(len(doc)):
            if doc[tok_id].lemma_ in uknames:
                try:
                    if doc[tok_id-1].pos_ in usual_posfeatures:
                        if doc[tok_id-2].pos_ in usual_posfeatures or doc[tok_id-2].text == '-':
                            if doc[tok_id-3].pos_ in usual_posfeatures or doc[tok_id-3].text == '-':
                                if doc[tok_id-4].pos_ in usual_posfeatures or doc[tok_id-4].text == '-':
                                    if doc[tok_id-5].pos_ in usual_posfeatures or doc[tok_id-5].text == '-':
                                        if doc[tok_id-6].pos_ in usual_posfeatures:
                                            ukfeatures_form_titles.append(doc[tok_id-6:tok_id].text)
                                        else:
                                            ukfeatures_form_titles.append(doc[tok_id-5:tok_id].text)
                                    else:
                                        ukfeatures_form_titles.append(doc[tok_id-4:tok_id].text)
                                else:
                                    ukfeatures_form_titles.append(doc[tok_id-3:tok_id].text)
                            else:
                                ukfeatures_form_titles.append(doc[tok_id-2:tok_id].text)
                        else:
                            ukfeatures_form_titles.append(doc[tok_id-1].text)
                except Exception:
                    answer = 'Там нет слова'

print(len(ukfeatures_form_titles))
ukfeatures_form_titles_set = set(ukfeatures_form_titles)
print(len(ukfeatures_form_titles_set))

1630
821


In [18]:
ukfeatures_set.update(ukfeatures_form_titles_set)
print(len(ukfeatures_set))

1158


In [19]:
ukfeatures = list(ukfeatures_set)
ukfeatures.sort()
#ukfeatures

Можно было бы использовать все наработки (+ поработать с леммами и регистром), но ограничимся малым.
# Выделяем сущности в отзывах
Делаем это примерно так, как выделяли features.

BIO формат - хорошо, но мы просто для каждого отзыва в отдельной колонке пропишем, какие упоминания встретились в отзыве.

'# Комментарий. Да, я понимаю, что код нельзя назвать красивым, но сейчас важно, чтоб он работал. 

In [20]:
rew_data['UkMentions'] = ""

In [27]:
usual_posfeatures = ['NOUN', 'ADJ', 'PROPN', 'NUM']

for index, row in rew_data.iterrows():
    ukmentions = []
    doc = nlp(str(row['reviewText']))
    for tok_id in range(len(doc)):
        if doc[tok_id].lemma_ in uknames:
            try:
                if doc[tok_id-1].pos_ in usual_posfeatures:
                    if doc[tok_id-2].pos_ in usual_posfeatures or doc[tok_id-2].text == '-':
                        if doc[tok_id-3].pos_ in usual_posfeatures or doc[tok_id-3].text == '-':
                            if doc[tok_id-4].pos_ in usual_posfeatures or doc[tok_id-4].text == '-':
                                if doc[tok_id-5].pos_ in usual_posfeatures or doc[tok_id-5].text == '-':
                                    if doc[tok_id-6].pos_ in usual_posfeatures:
                                        ukmentions.append(doc[tok_id-6:tok_id+1].text)
                                    else:
                                        ukmentions.append(doc[tok_id-5:tok_id+1].text)
                                else:
                                    ukmentions.append(doc[tok_id-4:tok_id+1].text)
                            else:
                                ukmentions.append(doc[tok_id-3:tok_id+1].text)
                        else:
                            ukmentions.append(doc[tok_id-2:tok_id+1].text)
                    else:
                        ukmentions.append(doc[tok_id-1:tok_id+1].text)
                else:
                    ukmentions.append(doc[tok_id].text)
            except Exception:
                answer = 'Там нет слова'
    rew_data['UkMentions'][index] = ukmentions

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [28]:
rew_data[['reviewText', 'UkMentions']]

,reviewText,UkMentions
0,"I hesitated at first, but nabbed the kit on a ...","[uke, cool ukes]"
1,Plain and simple...some of the frets are not p...,[uke]
2,"she loved it, won't stop playing",[it]
3,Thank you,[]
4,I decided that I wanted to learn to play the u...,"[ukulele, first ukulele, it, It, best ukulele]"
...,...,...
32332,I got this uke as a stand alone; it sounds ama...,"[uke, it]"
32333,This is what I want,[]
32334,The Kala KA-ZCT Tenor Gloss Ziricote uke sound...,"[KA-ZCT Tenor Gloss Ziricote uke, It, ukulele,..."
32335,This little ukulele has a booming voice! It t...,"[little ukulele, It, it, it, It, beautiful uku..."


# Посмотрим на соседей наших сущностей

In [79]:
# Будем работать с сокращённой таблицей, в которой в каждом отзыве есть упоминание.
pusto = []
short_rew = rew_data[rew_data['UkMentions'].map(lambda d: len(d)) > 0][['asin','reviewText', 'UkMentions']]
short_rew['left_n'] = ""
short_rew['right_n'] = ""


In [80]:
def lemmatizing(text):
    toks = word_tokenize(text)
    words = [w.lower() for w in toks if w.isalpha()]
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    return lemmas

In [81]:
short_rew['lemms'] = short_rew['reviewText'].progress_apply(lambda x: lemmatizing(x))

100%|██████████| 25139/25139 [01:09<00:00, 363.21it/s]


In [124]:
short_rew['lem_UkMentions'] = short_rew['UkMentions'].progress_apply(lambda x: [lemmatizing(i) for i in x])

100%|██████████| 25139/25139 [00:21<00:00, 1143.62it/s]


In [130]:
short_rew

,asin,reviewText,UkMentions,left_n,right_n,lemms,lem_UkMentions
0,0739079891,"I hesitated at first, but nabbed the kit on a ...","[uke, cool ukes]","[[the, uke], [of, cool, ukes]]","[[uke, is], [cool, ukes, out]]","[i, hesitated, at, first, but, nabbed, the, ki...","[[uke], [cool, ukes]]"
1,0739079891,Plain and simple...some of the frets are not p...,[uke],"[[the, uke]]","[[uke, does]]","[plain, and, simple, some, of, the, frets, are...",[[uke]]
2,0739079891,"she loved it, won't stop playing",[it],"[[loved, it]]","[[it, wo, stop, playing], [it, wo, stop, playi...","[she, loved, it, wo, stop, playing]",[[it]]
4,0739079891,I decided that I wanted to learn to play the u...,"[ukulele, first ukulele, it, It, best ukulele]","[[the, ukulele], [my, first, ukulele], [first,...","[[ukulele, as, a, beginner, i, knew, that, i, ...","[i, decided, that, i, wanted, to, learn, to, p...","[[ukulele], [first, ukulele], [it], [it], [bes..."
5,0739079891,I ordered this ukulele kit for my granddaughte...,"[ukulele, it, It, it]","[[this, ukulele], [loved, it], [loved, it], [l...","[[ukulele, kit, for, my, granddaughter, birthd...","[i, ordered, this, ukulele, kit, for, my, gran...","[[ukulele], [it], [it], [it]]"
...,...,...,...,...,...,...,...
32330,B01HEQ53OA,My daughter loves it!,[it],,,"[my, daughter, loves, it]",[[it]]
32331,B01HFKFWDW,"The Ukulele is absolutely gorgeous, and plays ...","[Ukulele, It, it, it, it, it, uke, it, it, it,...",,,"[the, ukulele, is, absolutely, gorgeous, and, ...","[[ukulele], [it], [it], [it], [it], [it], [uke..."
32332,B01HFKFWDW,I got this uke as a stand alone; it sounds ama...,"[uke, it]",,,"[i, got, this, uke, as, a, stand, alone, it, s...","[[uke], [it]]"
32334,B01HFKFWDW,The Kala KA-ZCT Tenor Gloss Ziricote uke sound...,"[KA-ZCT Tenor Gloss Ziricote uke, It, ukulele,...",,,"[the, kala, tenor, gloss, ziricote, uke, sound...","[[tenor, gloss, ziricote, uke], [it], [ukulele..."


In [131]:
# левый сосед
for index, row in short_rew.iterrows():
    left_n = []

    for tok_id in range(len(row['lemms'])):
        for ment_id in range(len(row['lem_UkMentions'])):
            try:
                if row['lemms'][tok_id] == row['lem_UkMentions'][ment_id][0]:
                    w_left = [row['lemms'][tok_id-1]]
                    w_left.extend(row['lem_UkMentions'][ment_id])
                    left_n.append(w_left)
            except Exception:
                answer = 'Там нет слова'
    short_rew['left_n'][index] = left_n

In [132]:
# правый сосед
for index, row in short_rew.iterrows():
    right_n = []

    for tok_id in range(len(row['lemms'])):
        for ment_id in range(len(row['lem_UkMentions'])):
            try:
                if row['lemms'][tok_id] == row['lem_UkMentions'][ment_id][-1]:
                    w_right = []
                    w_right.extend(row['lem_UkMentions'][ment_id])
                    w_right.append(row['lemms'][tok_id+1])
                    right_n.append(w_right)
            except Exception:
                answer = 'Там нет слова'
    short_rew['right_n'][index] = right_n

In [133]:
short_rew

,asin,reviewText,UkMentions,left_n,right_n,lemms,lem_UkMentions
0,0739079891,"I hesitated at first, but nabbed the kit on a ...","[uke, cool ukes]","[[the, uke], [of, cool, ukes]]","[[uke, is], [cool, ukes, out]]","[i, hesitated, at, first, but, nabbed, the, ki...","[[uke], [cool, ukes]]"
1,0739079891,Plain and simple...some of the frets are not p...,[uke],"[[the, uke]]","[[uke, does]]","[plain, and, simple, some, of, the, frets, are...",[[uke]]
2,0739079891,"she loved it, won't stop playing",[it],"[[loved, it]]","[[it, wo]]","[she, loved, it, wo, stop, playing]",[[it]]
4,0739079891,I decided that I wanted to learn to play the u...,"[ukulele, first ukulele, it, It, best ukulele]","[[the, ukulele], [my, first, ukulele], [first,...","[[ukulele, as], [first, ukulele, as], [best, u...","[i, decided, that, i, wanted, to, learn, to, p...","[[ukulele], [first, ukulele], [it], [it], [bes..."
5,0739079891,I ordered this ukulele kit for my granddaughte...,"[ukulele, it, It, it]","[[this, ukulele], [loved, it], [loved, it], [l...","[[ukulele, kit], [it, it], [it, it], [it, it],...","[i, ordered, this, ukulele, kit, for, my, gran...","[[ukulele], [it], [it], [it]]"
...,...,...,...,...,...,...,...
32330,B01HEQ53OA,My daughter loves it!,[it],"[[loves, it]]",[],"[my, daughter, loves, it]",[[it]]
32331,B01HFKFWDW,"The Ukulele is absolutely gorgeous, and plays ...","[Ukulele, It, it, it, it, it, uke, it, it, it,...","[[the, ukulele], [the, ukulele], [amazing, it]...","[[ukulele, is], [ukulele, is], [it, was], [it,...","[the, ukulele, is, absolutely, gorgeous, and, ...","[[ukulele], [it], [it], [it], [it], [it], [uke..."
32332,B01HFKFWDW,I got this uke as a stand alone; it sounds ama...,"[uke, it]","[[this, uke], [alone, it]]","[[uke, as], [it, sounds]]","[i, got, this, uke, as, a, stand, alone, it, s...","[[uke], [it]]"
32334,B01HFKFWDW,The Kala KA-ZCT Tenor Gloss Ziricote uke sound...,"[KA-ZCT Tenor Gloss Ziricote uke, It, ukulele,...","[[kala, tenor, gloss, ziricote, uke], [fantast...","[[tenor, gloss, ziricote, uke, sounds], [it, c...","[the, kala, tenor, gloss, ziricote, uke, sound...","[[tenor, gloss, ziricote, uke], [it], [ukulele..."


# Ранжируем

Тут я поняла, что я не знаю, что делать с моими NE, и начала почти с нуля.

In [134]:
from nltk.collocations import *

In [135]:
texts = short_rew['lemms'].to_list()

In [154]:
# Коллекция метрик
bigram_measures = nltk.collocations.BigramAssocMeasures()
# Экстрактор коллокаций
finder = BigramCollocationFinder.from_documents(texts)

In [169]:
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'instrument' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder.apply_ngram_filter(filter)

In [170]:
finder.score_ngrams(bigram_measures.likelihood_ratio)[:50]

[(('this', 'ukulele'), 10831.816696178113),
 (('this', 'uke'), 7410.24590845111),
 (('the', 'ukulele'), 3544.234657008523),
 (('this', 'instrument'), 2975.0722476951405),
 (('first', 'ukulele'), 2754.039890423317),
 (('an', 'instrument'), 2701.870914084869),
 (('the', 'uke'), 2395.2604825327803),
 (('the', 'instrument'), 2016.0160528049482),
 (('ukulele', 'for'), 1988.7650507428846),
 (('soprano', 'ukulele'), 1831.8357183987491),
 (('ukulele', 'is'), 1761.1403796785225),
 (('starter', 'uke'), 1746.3295476036924),
 (('musical', 'instrument'), 1642.494160126951),
 (('little', 'uke'), 1619.911334042907),
 (('other', 'ukes'), 1451.3941656785703),
 (('uke', 'for'), 1336.3471547862184),
 (('concert', 'ukulele'), 1307.6002267530912),
 (('starter', 'ukulele'), 1289.8987030877772),
 (('little', 'instrument'), 1230.9989872535443),
 (('concert', 'uke'), 1171.5044961520177),
 (('beautiful', 'instrument'), 1163.972610293445),
 (('stringed', 'instrument'), 1086.573108436624),
 (('tenor', 'ukulele'),

In [171]:
finder.apply_freq_filter(50)
finder.score_ngrams(bigram_measures.pmi)[:50]

[(('musical', 'instrument'), 6.485601163920148),
 (('stringed', 'instrument'), 6.308172854063955),
 (('other', 'ukes'), 6.147665855203929),
 (('expensive', 'ukes'), 5.980135368902001),
 (('these', 'ukes'), 5.4097402857997885),
 (('instrument', 'itself'), 5.125136785634833),
 (('starter', 'uke'), 5.079451327603326),
 (('baritone', 'uke'), 4.936292386807754),
 (('real', 'instrument'), 4.895795368542348),
 (('sounding', 'instrument'), 4.820418325947902),
 (('uke', 'itself'), 4.745110449495016),
 (('an', 'instrument'), 4.549498816640899),
 (('baritone', 'ukulele'), 4.536180700613617),
 (('concert', 'uke'), 4.49572727414872),
 (('ukulele', 'itself'), 4.4742817802458426),
 (('tenor', 'uke'), 4.4497444563106825),
 (('soprano', 'ukulele'), 4.427389796056932),
 (('starter', 'ukulele'), 4.421305916633763),
 (('inexpensive', 'instrument'), 4.299190880000765),
 (('beautiful', 'instrument'), 4.268523448383434),
 (('concert', 'ukulele'), 4.256262775808889),
 (('soprano', 'uke'), 4.239007290559655),


In [175]:
finder.apply_freq_filter(40)
finder.score_ngrams(bigram_measures.dice)[:50]

[(('this', 'ukulele'), 0.14273634364958565),
 (('this', 'uke'), 0.1118144960523508),
 (('other', 'ukes'), 0.09213627598028712),
 (('an', 'instrument'), 0.088038134962014),
 (('first', 'ukulele'), 0.0658402946195994),
 (('this', 'instrument'), 0.061876825129567756),
 (('the', 'ukulele'), 0.06040834353857582),
 (('ukulele', 'for'), 0.05763952424519671),
 (('little', 'uke'), 0.05460164835164835),
 (('ukulele', 'is'), 0.05356931688280973),
 (('little', 'instrument'), 0.052307229821133325),
 (('beautiful', 'instrument'), 0.049694933239013174),
 (('musical', 'instrument'), 0.04879694175848887),
 (('starter', 'uke'), 0.04678024833958995),
 (('expensive', 'ukes'), 0.04540295119182747),
 (('uke', 'for'), 0.04502317369234165),
 (('the', 'uke'), 0.04440446584475369),
 (('soprano', 'ukulele'), 0.04194871794871795),
 (('first', 'uke'), 0.04060913705583756),
 (('quality', 'instrument'), 0.039443949873313705),
 (('uke', 'is'), 0.03871873013088948),
 (('ukulele', 'and'), 0.03767408470926059),
 (('conc

Итого: мне больше всего понравился результат PMI с фильтром. Получаются более специфические биграммы. (Мы знаем, что PMI "переоценивает" редкие слова.) Но гораздо больше мне нравились мои NE из предыдущей части задания.

# Посмотрим NE для разных брендов

In [202]:
# Объединили тексты с брендами
short_rew = short_rew.join(meta_df.set_index('asin'), on='asin')

In [206]:
# Возьмём 5 довольно популярных брендов и посмотрим, что про них пишут.

Lanikai_texts = short_rew.loc[short_rew['brand'] == 'Lanikai']['lemms'].to_list()
Oscar_texts = short_rew.loc[short_rew['brand'] == 'Oscar Schmidt']['lemms'].to_list()
Kohala_texts = short_rew.loc[short_rew['brand'] == 'Kohala']['lemms'].to_list()
Mahalo_texts = short_rew.loc[short_rew['brand'] == 'Mahalo']['lemms'].to_list()
Luna_texts = short_rew.loc[short_rew['brand'] == 'Luna Guitars']['lemms'].to_list()

In [212]:
print('Lanikai')
finder_Lanikai = BigramCollocationFinder.from_documents(Lanikai_texts)
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder_Lanikai.apply_ngram_filter(filter)
finder_Lanikai.apply_freq_filter(7)
finder_Lanikai.score_ngrams(bigram_measures.pmi)[:6]

Lanikai


[(('other', 'ukes'), 5.802085484795196),
 (('starter', 'uke'), 5.5252941811387295),
 (('baritone', 'uke'), 5.315371111764495),
 (('ukulele', 'bundle'), 4.869592304153475),
 (('concert', 'uke'), 4.807887363721058),
 (('baritone', 'ukulele'), 4.757382800567445)]

In [213]:
print('Oscar Schmidt')
finder_Oscar = BigramCollocationFinder.from_documents(Oscar_texts)
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder_Oscar.apply_ngram_filter(filter)
finder_Oscar.apply_freq_filter(7)
finder_Oscar.score_ngrams(bigram_measures.pmi)[:6]

Oscar Schmidt


[(('ukulele', 'club'), 6.563424936838851),
 (('other', 'ukes'), 6.473979614878354),
 (('ukulele', 'itself'), 4.978462436117697),
 (('baritone', 'uke'), 4.731252997445587),
 (('uke', 'itself'), 4.627965189033562),
 (('soprano', 'uke'), 4.610958763727872)]

In [215]:
print('Kohala')
finder_Kohala = BigramCollocationFinder.from_documents(Kohala_texts)
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder_Kohala.apply_ngram_filter(filter)
finder_Kohala.apply_freq_filter(7)
finder_Kohala.score_ngrams(bigram_measures.pmi)[:6]

Kohala


[(('first', 'ukulele'), 4.689986817163966),
 (('this', 'ukulele'), 3.6418712349378612),
 (('this', 'uke'), 3.6337865048061726),
 (('uke', 'for'), 2.580585727544717),
 (('my', 'ukulele'), 2.2720605127968945),
 (('ukulele', 'for'), 2.024769572483075)]

In [216]:
print('Mahalo')
finder_Mahalo = BigramCollocationFinder.from_documents(Mahalo_texts)
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder_Mahalo.apply_ngram_filter(filter)
finder_Mahalo.apply_freq_filter(7)
finder_Mahalo.score_ngrams(bigram_measures.pmi)[:6]

Mahalo


[(('little', 'uke'), 4.379539142838929),
 (('first', 'ukulele'), 4.077812434590166),
 (('this', 'ukulele'), 3.686446572058749),
 (('this', 'uke'), 3.5592583721121187),
 (('little', 'ukulele'), 2.6199618517723504),
 (('ukulele', 'is'), 2.4902187184015787)]

In [217]:
print('Luna Guitars')
finder_Luna = BigramCollocationFinder.from_documents(Luna_texts)
filter = lambda *w: 'ukulele' not in w and 'uke' not in w and 'ukes' not in w and 'eku' not in w and 'cavaquinho' not in w and 'machete' not in w and 'cavaco' not in w and 'cavacolele' not in w and 'timple' not in w and 'taropatch' not in w
finder_Luna.apply_ngram_filter(filter)
finder_Luna.apply_freq_filter(7)
finder_Luna.score_ngrams(bigram_measures.pmi)[:6]

Luna Guitars


[(('other', 'ukes'), 6.756307436606946),
 (('beginner', 'uke'), 4.69181962345249),
 (('concert', 'uke'), 4.414285647923583),
 (('concert', 'ukulele'), 3.9645078463853007),
 (('first', 'ukulele'), 3.8510720825971934),
 (('first', 'uke'), 3.6728186615224363)]

Не очень чувствуются различия между брендами. Но думаю, что это оттого, что у них у всех примерно одинаковых типов укулеле.

Было бы здорово отфильтровать биграммы,чтоб соседями были, например, только ADJ. А ещё можно были три-четыре-итд-граммы добавить.